In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [44]:
import json
import datetime 
fid = open("data/trains.json")
data_raw = json.load(fid)
fid.close()
start_time = datetime.datetime.strptime("2017-10-17", "%Y-%m-%d")
end_time = datetime.datetime.strptime("2017-12-1", "%Y-%m-%d")
feats_raw = np.zeros([1080,1+7+24])
labs = np.zeros([1080,33])
for data in data_raw:
    data_time = datetime.datetime.strptime(data["date"], "%Y-%m-%d:%H")
    if data_time >= start_time:
        n = int((data_time - start_time).total_seconds()/(60*60))
        feats_raw[n][0] = data["holiday"]
        feats_raw[n][data["week"]] = 1
        feats_raw[n][data_time.hour+8] = 1
        labs[n][data["place"]-1] = data["people"]

In [48]:
HIDDEN_SIZE = 30                            # LSTM中隐藏节点的个数。
NUM_LAYERS = 2                              # LSTM的层数。
TIMESTEPS = 24*7                            # 循环神经网络的训练序列长度。
TRAINING_STEPS = 10000                      # 训练轮数。
BATCH_SIZE = 32                             # batch大小。

In [130]:
train_feats = np.array([])
train_labs = np.array([])
for i in range(6):
    if i == 0:
        train_feats = np.reshape(feats_raw[TIMESTEPS*i:TIMESTEPS*(i+1)], [-1])
        train_labs = np.reshape(labs[TIMESTEPS*i:TIMESTEPS*(i+1)], [-1])
    else:
        train_feats = np.vstack([train_feats, np.reshape(feats_raw[TIMESTEPS*i:TIMESTEPS*(i+1)], [-1])])
        train_labs = np.vstack([train_labs, np.reshape(labs[TIMESTEPS*i:TIMESTEPS*(i+1)], [-1])])
val_feats = np.reshape(feats_raw[TIMESTEPS*i:TIMESTEPS*(i+1)], [-1])
val_labs = np.reshape(labs[TIMESTEPS*i:TIMESTEPS*(i+1)], [-1])

In [149]:
val_labs.shape = [1,-1,1]
val_labs.shape

ValueError: can only specify one unknown dimension

In [134]:
def lstm_model(X, y, is_training):
    cell = tf.nn.rnn_cell.MultiRNNCell([
        tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE) 
        for _ in range(NUM_LAYERS)])

    outputs, _ = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    #output = outputs[:, -1, :]
    output = outputs[:, -1, :]

    predictions = tf.contrib.layers.fully_connected(
        output, 1, activation_fn=None)
    
    if not is_training:
        return predictions, None, None

    loss = tf.losses.mean_squared_error(labels=y, predictions=predictions)

    train_op = tf.contrib.layers.optimize_loss(
        loss, tf.train.get_global_step(),
        optimizer="Adagrad", learning_rate=0.1)
    return predictions, loss, train_op

In [83]:
def run_eval(sess, test_X, test_y):
    ds = tf.data.Dataset.from_tensor_slices((test_X, test_y))
    ds = ds.batch(1)
    X, y = ds.make_one_shot_iterator().get_next()
    
    with tf.variable_scope("model", reuse=True):
        prediction, _, _ = lstm_model(X, [0.0], False)
    
    predictions = []
    labels = []
    for i in range(TESTING_EXAMPLES):
        p, l = sess.run([prediction, y])
        predictions.append(p)
        labels.append(l)

    predictions = np.array(predictions).squeeze()
    labels = np.array(labels).squeeze()
    rmse = np.sqrt(((predictions - labels) ** 2).mean(axis=0))
    print("Root Mean Square Error is: %f" % rmse)
    
    plt.figure()
    plt.plot(predictions[1], label='predictions')
    plt.plot(labels[1], label='real_sin')
    plt.legend()
    plt.show()

In [138]:
ds = tf.data.Dataset.from_tensor_slices((train_feats, train_labs))

In [139]:
ds = ds.repeat().shuffle(1000).batch(BATCH_SIZE)

In [141]:
X, y = ds.make_one_shot_iterator().get_next()

In [143]:
with tf.variable_scope("model"):
    _, loss, train_op = lstm_model(X, y, True)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 测试在训练之前的模型效果。
    print("Evaluate model before training.")
    run_eval(sess, test_X, test_y)
    
    # 训练模型。
    for i in range(TRAINING_STEPS):
        _, l = sess.run([train_op, loss])
        if i % 1000 == 0:
            print("train step: " + str(i) + ", loss: " + str(l))
    
    # 使用训练好的模型对测试数据进行预测。
    print("Evaluate model after training.")
    run_eval(sess, test_X, test_y)

ValueError: Shape (5376, ?) must have rank at least 3

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((train_feats, train_labs))
ds = ds.repeat().shuffle(1000).batch(BATCH_SIZE)
X, y = ds.make_one_shot_iterator().get_next()

# 定义模型，得到预测结果、损失函数，和训练操作。
with tf.variable_scope("model"):
    _, loss, train_op = lstm_model(X, y, True)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # 测试在训练之前的模型效果。
    print("Evaluate model before training.")
    run_eval(sess, test_X, test_y)
    
    # 训练模型。
    for i in range(TRAINING_STEPS):
        _, l = sess.run([train_op, loss])
        if i % 1000 == 0:
            print("train step: " + str(i) + ", loss: " + str(l))
    
    # 使用训练好的模型对测试数据进行预测。
    print("Evaluate model after training.")
    run_eval(sess, test_X, test_y)